# Thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import random


## 1. Cấu hình + Khởi tạo driver

In [2]:
# Cấu hình dải trang cần crawl
START_PAGE = 399
END_PAGE = 400  # crawl từ 348 tới 400 (bao gồm)

CSV_PATH = "phongtro.csv"


def random_sleep(a=0.5, b=1.0):
    """
    Ngủ ngẫu nhiên trong khoảng [a, b] giây.
    (Tiện ích dùng chung)
    """
    time.sleep(random.uniform(a, b))


def create_driver():
    """
    Khởi tạo trình duyệt Chrome với các options cần thiết.
    (Thành viên 1 phụ trách)
    """
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Bỏ comment để ẩn trình duyệt
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    )

    driver = webdriver.Chrome(options=chrome_options)
    return driver


## 2. Lấy danh sách link từng trang

In [3]:
def get_links_for_page(driver, page):
    """
    Vào trang ?page=... và lấy tất cả link bài đăng.
    Trả về: list các URL bài đăng.
    """
    url = f"https://phongtro123.com/?page={page}"
    print(f"\nĐang crawl trang {page}...")
    driver.get(url)

    links = []

    try:
        # Đợi danh sách bài đăng load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.post__listing li"))
        )
        random_sleep(0.5, 1.0)

        # Lấy tất cả link bài đăng 
        posts = driver.find_elements(By.CSS_SELECTOR, "h3.fs-6 a.line-clamp-2")
        links = [p.get_attribute("href") for p in posts if p.get_attribute("href")]

        print(f"Tìm thấy {len(links)} bài đăng")

        # Nếu không tìm thấy thì sẽ thử selector fallback
        if len(links) == 0:
            print("Không tìm thấy bài nào, thử selector khác...")
            posts = driver.find_elements(
                By.CSS_SELECTOR, "li.d-flex a[href*='phongtro123.com']"
            )
            raw_links = [
                p.get_attribute("href")
                for p in posts
                if p.get_attribute("href")
                and "/tinh-thanh/" not in p.get_attribute("href")
            ]
            links = list(set(raw_links))
            print(f"Fallback: Tìm thấy {len(links)} bài")
    except Exception as e:
        print(f"Lỗi khi load trang {page}: {e}")

    return links


## 3. Crawl thông tin cơ bản

In [4]:
def extract_basic_info(driver, link, idx, total_in_page):
    """
    Vào 1 bài đăng và lấy các thông tin cơ bản:
    - title, address, price, area
    - ngaydang
    - description (mô tả)
    Trả về dict với các key trên + "url".
    """
    print(f"  [{idx}/{total_in_page}] Đang crawl: {link[:60]}...")
    driver.get(link)
    random_sleep(0.5, 1.0)

    # --- Lấy tiêu đề ---
    try:
        title = driver.find_element(By.TAG_NAME, "h1").text.strip()
    except:
        title = "N/A"

    # --- Lấy địa chỉ ---
    address = "N/A"
    try:
        # Cách 1: Tìm dòng có text "Địa chỉ:"
        address_row = driver.find_element(
            By.XPATH, "//td//div[contains(text(), 'Địa chỉ:')]"
        )
        address = (
            address_row.find_element(By.XPATH, "../..")
            .find_element(By.XPATH, "./td[2]")
            .text.strip()
        )
    except:
        try:
            # Cách 2: Fallback - dùng selector cũ
            elem = driver.find_element(By.XPATH, "//td[contains(text(), 'Đường')]")
            row = elem.find_element(By.XPATH, "..")
            addr_text = row.text.strip()
            addr_text = addr_text.replace("Đường:", "").strip()
            address = addr_text
        except:
            pass

    # --- Lấy giá ---
    try:
        price = driver.find_element(
            By.CSS_SELECTOR, "span.text-green.fs-5.fw-bold"
        ).text.strip()
    except:
        price = "N/A"

    # --- Lấy diện tích ---
    try:
        area = driver.find_element(
            By.CSS_SELECTOR,
            "div.justify-content-between > div.d-flex > span:nth-of-type(3)",
        ).text.strip()
    except:
        area = "N/A"

    # --- LẤY NGÀY ĐĂNG ---
    ngaydang = "N/A"
    try:
        ngaydang_row = driver.find_element(
            By.XPATH, "//td//div[contains(text(), 'Ngày đăng:')]"
        )
        time_element = (
            ngaydang_row.find_element(By.XPATH, "../..")
            .find_element(By.TAG_NAME, "time")
        )

        ngaydang_attr = time_element.get_attribute("title")
        if ngaydang_attr and ngaydang_attr.strip():
            ngaydang = ngaydang_attr.strip()
        else:
            ngaydang_text = time_element.text.strip()
            if ngaydang_text:
                ngaydang = ngaydang_text
    except:
        pass

    # --- LẤY THÔNG TIN MÔ TẢ ---
    description = "N/A"
    try:
        desc_container = driver.find_element(
            By.CSS_SELECTOR, "div.border-bottom.pb-3.mb-4"
        )
        paragraphs = desc_container.find_elements(By.TAG_NAME, "p")
        desc_lines = []

        for p in paragraphs:
            text = p.text.strip()
            if text:
                desc_lines.append(text)

        if desc_lines:
            description = " | ".join(desc_lines)

        if not description.strip():
            description = "N/A"
    except Exception as e:
        if idx == 1:
            print(f"Không lấy được mô tả: {e}")

    print(f"{title[:50]}...")

    basic_info = {
        "title": title,
        "address": address,
        "price": price,
        "area": area,
        "ngaydang": ngaydang,
        "thongtinmota": description,
        "url": link,
    }

    return basic_info


## 4. Crawl features + Ghép CSV + Vòng lặp chính

In [5]:
def extract_features(driver, idx_in_page):
    """
    Đọc các feature phòng trọ:
    - Đầy đủ nội thất, Có gác, Có kệ bếp, ...
    Trả về dict với 0/1 cho từng feature.
    (Thành viên 4 phụ trách)
    """
    feature_map = {
        "Đầy đủ nội thất": 0,
        "Có gác": 0,
        "Có kệ bếp": 0,
        "Có máy lạnh": 0,
        "Có máy giặt": 0,
        "Có tủ lạnh": 0,
        "Có thang máy": 0,
        "Không chung chủ": 0,
        "Giờ giấc tự do": 0,
        "Có hầm để xe": 0,
    }

    try:
        feature_divs = driver.find_elements(
            By.XPATH,
            "//div[contains(@class, 'text-body') and contains(@class, 'd-flex')]",
        )
        for div in feature_divs:
            try:
                text = div.text.strip()
                if text not in feature_map:
                    continue

                has_green_icon = False
                parent_has_opacity = False

                try:
                    icon = div.find_element(By.TAG_NAME, "i")
                    icon_class = icon.get_attribute("class") or ""
                    has_green_icon = "green" in icon_class
                except:
                    pass

                style = div.get_attribute("style") or ""
                parent_has_opacity = ("opacity" in style) or (
                    "bs-text-opacity" in style
                )

                if has_green_icon and not parent_has_opacity:
                    feature_map[text] = 1
            except:
                continue
    except Exception as e:
        if idx_in_page == 1:
            print(f"    Không lấy được features: {e}")

    return feature_map


def build_csv_row(basic_info, feature_map):
    """
    Ghép basic_info (phần 3) + feature_map (phần này) thành 1 dòng CSV.
    (Thành viên 4 phụ trách)
    """
    return [
        basic_info.get("title", "N/A"),
        basic_info.get("address", "N/A"),
        basic_info.get("price", "N/A"),
        basic_info.get("area", "N/A"),
        feature_map.get("Đầy đủ nội thất", 0),
        feature_map.get("Có gác", 0),
        feature_map.get("Có kệ bếp", 0),
        feature_map.get("Có máy lạnh", 0),
        feature_map.get("Có máy giặt", 0),
        feature_map.get("Có tủ lạnh", 0),
        feature_map.get("Có thang máy", 0),
        feature_map.get("Không chung chủ", 0),
        feature_map.get("Giờ giấc tự do", 0),
        feature_map.get("Có hầm để xe", 0),
        basic_info.get("ngaydang", "N/A"),
        basic_info.get("thongtinmota", "N/A"),
        basic_info.get("url", "N/A"),
    ]

def write_csv_header(writer):
    """
    Ghi dòng header vào file CSV.
    (Thành viên 4 phụ trách)
    """
    writer.writerow(
        [
            "title", "address", "price", "area",
            "noithat", "gac", "kebep", "maylanh", "maygiat",
            "tulanh", "thangmay", "chungchu", "giotu", "hamxe",
            "ngaydang",      
            "thongtinmota",  
            "url",
        ]
    )


def run_crawler(driver, writer):
    """
    Vòng lặp chính:
    - Lặp qua các trang (gọi hàm thành viên 2)
    - Lặp qua các bài (gọi hàm thành viên 3 + 4)
    - Ghi vào CSV, đếm tổng số bài.
    (Thành viên 4 phụ trách)
    """
    total_rows = 0

    for page in range(START_PAGE, END_PAGE + 1):
        links = get_links_for_page(driver, page)
        if not links:
            continue

        for idx, link in enumerate(links, 1):
            try:
                basic_info = extract_basic_info(driver, link, idx, len(links))  
                feature_map = extract_features(driver, idx)                   
                row = build_csv_row(basic_info, feature_map)                   
                writer.writerow(row)
                total_rows += 1
            except Exception as e:
                print(f"    Lỗi khi crawl bài: {e}")
                continue

    return total_rows


## 5. Main

In [6]:
def main():
    """
    Hàm main: tạo driver, mở file CSV, gọi run_crawler rồi đóng driver.
    """
    driver = create_driver()
    try:
        with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as csv_file:
            writer = csv.writer(csv_file)
            write_csv_header(writer)  
            total_rows = run_crawler(driver, writer)  

        print("\n" + "=" * 70)
        print(f"HOÀN TẤT! Đã ghi {total_rows} bài vào file '{CSV_PATH}'")
        print("=" * 70)
    except Exception as e:
        print(f"\nLỖI NGHIÊM TRỌNG: {e}")
    finally:
        driver.quit()
        print("Đã đóng trình duyệt")


In [7]:
if __name__ == "__main__":
    main()



Đang crawl trang 399...
Tìm thấy 20 bài đăng
  [1/20] Đang crawl: https://phongtro123.com/gac-cao-2m-full-do-dac-ngay-nguyen-d...
GÁC CAO 2M, FULL ĐỒ ĐẠC NGAY NGUYỄN DUY TRINH, BÌN...
  [2/20] Đang crawl: https://phongtro123.com/gia-sieu-uu-dai-studio-quan-2-tu-4x-...
GIÁ SIÊU ƯU ĐÃI STUDIO QUẬN 2 từ 4x 5x...
  [3/20] Đang crawl: https://phongtro123.com/nha-tro-dong-mai-yen-nghia-bien-gian...
Nhà trọ Đồng Mai - Nhà có 8 phòng trọ cần cho thuê...
  [4/20] Đang crawl: https://phongtro123.com/chi-5-trieu-phong-moi-xay-duong-khue...
CHỈ 5 TRIỆU - PHÒNG MỚI XÂY - DƯƠNG KHUÊ - CẦU GIẤ...
  [5/20] Đang crawl: https://phongtro123.com/phong-tro-tan-binh-gan-san-bay-khu-c...
Phòng trọ Tân Bình gần sân bay khu cán bộ THĂNG LO...
  [6/20] Đang crawl: https://phongtro123.com/phong-tro-trong-nha-rieng-khu-vuc-ph...
Phòng tro trong nhà riêng khu vuc Phú Nhuan...
  [7/20] Đang crawl: https://phongtro123.com/cho-thue-1-phong-sub-chung-cu-can-ho...
Cho thuê 1 phòng Sub. Chung cư Căn hộ Dragon Hill ...
